In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
import requests
import pickle
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
dataset = pd.read_csv('/content/drive/MyDrive/SNF_IITK/model.csv')

In [3]:
## functions to clean the news text
def removeTag(text):
    remove = re.compile(r'')
    return re.sub(remove, '', text)

#removes special characters
def removeSpec_char(text):
    cleanString = text.replace("\n", " ")
    return cleanString

def lowerCase(text):
    return text.lower()

#removes punctuation
def remove_punc(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(text)
    return new_words

#removes stop words
def removeStop(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [x for x in words if x not in stop_words]

# lemmatizes the words
def lemmaWord(text):
    wordnet = WordNetLemmatizer()
    return " ".join([wordnet.lemmatize(word) for word in text])

In [7]:
dataset['news'] = dataset['news'].apply(removeTag)
dataset['news'] = dataset['news'].apply(removeSpec_char)
dataset['news'] = dataset['news'].apply(lowerCase)
dataset['news'] = dataset['news'].apply(remove_punc)
dataset['news'] = dataset['news'].apply(lemmaWord)
dataset['news'] = dataset['news'].apply(removeStop)
dataset['news'] = dataset['news'].apply(lemmaWord)

In [8]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
dataset['label']= label_encoder.fit_transform(dataset['subcateg'])
dataset['label'].unique()

array([12,  1, 10,  3,  6,  4,  8,  0,  2,  7,  9, 11,  5])

In [9]:
x = np.array(dataset.iloc[:,0].values)
y = np.array(dataset.label.values)
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(dataset.news).toarray()
print("X.shape = ",x.shape)
print("y.shape = ",y.shape)

X.shape =  (23672, 5000)
y.shape =  (23672,)


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0, shuffle = True)
print(len(x_train))
print(len(x_test))

21304
2368


In [ ]:
#One v Rest classifier with random forest decision function
rf = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
model = OneVsRestClassifier(rf)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

In [12]:
accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)
# Get precision, recall, f1 scores
precision, recall, f1score, support = score(y_test, y_pred, average='micro')
print(f'Accuracy: {accuracy}')
print(f'Precision : {precision}')
print(f'Recall : {recall}')
print(f'F1-score : {f1score}')

Accuracy: 72.17
Precision : 0.721706081081081
Recall : 0.721706081081081
F1-score : 0.721706081081081


In [ ]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))